In [ ]:
import pandas as pd
from stage import fullPrint, getStage
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
import numpy as np
import pickle
import swifter
import os

nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

PEAK = 13
allList = []
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
allList = (nasdaqList + nyseList)

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
stockDataTickerList = []
now = datetime.now()

In [ ]:
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
        dfBuy = df
        df['Dates'] = pd.to_datetime(df.index)
        df = df[df['Dates'].dt.weekday == 4]
        df = df.drop('Dates', axis = 1)
        df.to_pickle('stockData/nysenasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
        dfBuy['Dates'] = pd.to_datetime(dfBuy.index)
        dfBuy = dfBuy[dfBuy['Dates'].dt.weekday == 0]
        dfBuy = dfBuy.drop('Dates', axis = 1)
        dfBuy.to_pickle('stockData/nysenasdaq/'+ticker+'Buy.pkl')
        stockDataTickerList.append(ticker)
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
except:
    print("Something went wrong")

In [ ]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def calculateSpyDfSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,8]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [ ]:
spDf = yf.get_data('^GSPC', start_date="1995-01-06",end_date = now, index_as_date = True)
spDf['close'] = spDf['close'].rolling(window=5).apply(lambda x: np.average(x))
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSpyDfSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf.to_pickle("stockData/Spy.pkl")

In [ ]:
for ticker in stockDataTickerList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [ ]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= now, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [ ]:
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [ ]:
now = datetime.now()
monday = now - timedelta(days = now.weekday())
now = now.strftime('%Y-%m-%d')
df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
df['Dates'] = pd.to_datetime(df.index)
df = df[df['Dates'].dt.weekday == 4]
df = df.drop('Dates', axis = 1)
df

In [ ]:
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(stockDataTickerList, f)

In [ ]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [2]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            #print(df.at[date,'MA30Slope'],df.at[date,'RS'])
            #!!!!!!!! or for new params
            #!!!!!!!!
            #!!!!!!!!
            #!!!!!!!!
            if  df.at[date,'MA30Slope'] > 1.0013386634337298 and df.loc[date,'RS'] > 0.07551020458494859:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1
removeList = []
for ticker in nasdaqList:
    try:
        df = getStage(ticker,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
        if df.empty:
            removeList.append(ticker)
    except:
        removeList.append(ticker)
print(removeList)

['ARCKW', 'AVHIW', 'BPACW', 'CINGW', 'DXR', 'EQRXW', 'ESACW', 'MCAAW', 'ONYXW', 'PCCTW', 'STFC', 'TOACW', 'WEN', 'WINA']


In [3]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)

In [28]:
import pandas as pd
df = pd.read_pickle("stockData/nasdaq.pkl")
print(df['USEG'])

XLE
